In [1]:
from imutils.video import VideoStream
import argparse
import imutils
import time
import cv2
import sys
import numpy as np
import os

In [32]:
import cv2
import numpy as np

all_camera_idx_available = []

for camera_idx in range(-100, 900):
    cap = cv2.VideoCapture(camera_idx)
    if cap.isOpened():
        print(f'Camera index available: {camera_idx}')
        all_camera_idx_available.append(camera_idx)
        cap.release()

Camera index available: 0


In [25]:
vs.stop()

In [2]:
arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
arucoParams = cv2.aruco.DetectorParameters_create()
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)

[INFO] starting video stream...


In [4]:
while True:
    frame = vs.read()

    image = imutils.resize(frame, width = 1000)
    (corners, ids, rejected) = cv2.aruco.detectMarkers(image, arucoDict, parameters=arucoParams)
    if len(corners) > 0:
        # flatten the ArUco IDs list
        ids = ids.flatten()
        # loop over the detected ArUCo corners
        for (markerCorner, markerID) in zip(corners, ids):
            # extract the marker corners (which are always returned in
            # top-left, top-right, bottom-right, and bottom-left order)
            #print(markerCorner)
            print(markerCorner)
            #corners = markerCorner.reshape((4, 1))
            (topLeft, topRight, bottomRight, bottomLeft) = markerCorner[0]
            # convert each of the (x, y)-coordinate pairs to integers
            topRight = (int(topRight[0]), int(topRight[1]))
            bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
            bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
            topLeft = (int(topLeft[0]), int(topLeft[1]))

            # draw the bounding box of the ArUCo detection
            cv2.line(image, topLeft, topRight, (0, 255, 0), 2)
            cv2.line(image, topRight, bottomRight, (0, 255, 0), 2)
            cv2.line(image, bottomRight, bottomLeft, (0, 255, 0), 2)
            cv2.line(image, bottomLeft, topLeft, (0, 255, 0), 2)
            # compute and draw the center (x, y)-coordinates of the ArUco
            # marker
            cX = int((topLeft[0] + bottomRight[0]) / 2.0)
            cY = int((topLeft[1] + bottomRight[1]) / 2.0)
            cv2.circle(image, (cX, cY), 4, (0, 0, 255), -1)
            # draw the ArUco marker ID on the image
            cv2.putText(image, str(markerID),
                (topLeft[0], topLeft[1] - 15), cv2.FONT_HERSHEY_SIMPLEX,
                0.5, (0, 255, 0), 2)
            print("[INFO] ArUco marker ID: {}".format(markerID))
    # show the output image
    cv2.imshow("Image", image)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord("q"):
        break
cv2.destroyAllWindows()
vs.stop()

[[[154. 363.]
  [277. 370.]
  [291. 469.]
  [154. 478.]]]
[INFO] ArUco marker ID: 1
[[[154. 363.]
  [277. 370.]
  [291. 469.]
  [154. 478.]]]
[INFO] ArUco marker ID: 1
[[[154. 363.]
  [277. 370.]
  [291. 469.]
  [154. 478.]]]
[INFO] ArUco marker ID: 1
[[[322. 339.]
  [439. 354.]
  [424. 473.]
  [301. 460.]]]
[INFO] ArUco marker ID: 2
[[[322. 339.]
  [439. 354.]
  [424. 473.]
  [301. 460.]]]
[INFO] ArUco marker ID: 2
[[[322. 339.]
  [439. 354.]
  [424. 473.]
  [301. 460.]]]
[INFO] ArUco marker ID: 2
[[[311. 345.]
  [435. 354.]
  [423. 479.]
  [298. 470.]]]
[INFO] ArUco marker ID: 2
[[[ 35. 311.]
  [147. 329.]
  [146. 446.]
  [ 32. 426.]]]
[INFO] ArUco marker ID: 0
[[[ 32. 447.]
  [146. 465.]
  [164. 550.]
  [ 56. 528.]]]
[INFO] ArUco marker ID: 3
[[[ 32. 447.]
  [146. 465.]
  [164. 550.]
  [ 56. 528.]]]
[INFO] ArUco marker ID: 3
[[[ 32. 447.]
  [146. 465.]
  [164. 550.]
  [ 56. 528.]]]
[INFO] ArUco marker ID: 3
[[[ 61. 435.]
  [178. 458.]
  [178. 556.]
  [ 65. 531.]]]
[INFO] ArUco marke

[[[140. 360.]
  [251. 373.]
  [258. 479.]
  [147. 469.]]]
[INFO] ArUco marker ID: 6
[[[264. 237.]
  [386. 246.]
  [381. 373.]
  [262. 360.]]]
[INFO] ArUco marker ID: 4
[[[138. 227.]
  [251. 237.]
  [250. 359.]
  [139. 347.]]]
[INFO] ArUco marker ID: 3
[[[415. 110.]
  [542. 125.]
  [532. 245.]
  [401. 233.]]]
[INFO] ArUco marker ID: 2
[[[275.  98.]
  [402. 109.]
  [387. 233.]
  [265. 224.]]]
[INFO] ArUco marker ID: 1
[[[143.  91.]
  [262.  97.]
  [252. 223.]
  [138. 214.]]]
[INFO] ArUco marker ID: 0
[[[262. 373.]
  [381. 384.]
  [385. 493.]
  [272. 480.]]]
[INFO] ArUco marker ID: 7
[[[139. 360.]
  [250. 371.]
  [259. 479.]
  [149. 469.]]]
[INFO] ArUco marker ID: 6
[[[264. 236.]
  [386. 245.]
  [380. 372.]
  [262. 359.]]]
[INFO] ArUco marker ID: 4
[[[138. 227.]
  [251. 235.]
  [250. 358.]
  [139. 347.]]]
[INFO] ArUco marker ID: 3
[[[276.  98.]
  [402. 108.]
  [387. 231.]
  [265. 223.]]]
[INFO] ArUco marker ID: 1
[[[144.  91.]
  [263.  97.]
  [251. 222.]
  [138. 214.]]]
[INFO] ArUco marke

[[[162. 373.]
  [252. 387.]
  [253. 503.]
  [154. 487.]]]
[INFO] ArUco marker ID: 6
[[[265. 251.]
  [388. 257.]
  [386. 381.]
  [264. 375.]]]
[INFO] ArUco marker ID: 4
[[[160. 245.]
  [253. 251.]
  [252. 374.]
  [162. 360.]]]
[INFO] ArUco marker ID: 3
[[[157. 240.]
  [262. 247.]
  [258. 375.]
  [148. 363.]]]
[INFO] ArUco marker ID: 3
[[[172. 104.]
  [267. 103.]
  [265. 234.]
  [158. 224.]]]
[INFO] ArUco marker ID: 0
[[[274. 396.]
  [399. 405.]
  [400. 513.]
  [277. 512.]]]
[INFO] ArUco marker ID: 7
[[[150. 246.]
  [266. 255.]
  [260. 384.]
  [145. 371.]]]
[INFO] ArUco marker ID: 3
[[[151. 391.]
  [266. 406.]
  [266. 521.]
  [149. 512.]]]
[INFO] ArUco marker ID: 6
[[[280. 263.]
  [408. 269.]
  [403. 399.]
  [280. 391.]]]
[INFO] ArUco marker ID: 4
[[[152. 254.]
  [267. 263.]
  [265. 390.]
  [151. 376.]]]
[INFO] ArUco marker ID: 3
[[[282. 408.]
  [406. 414.]
  [409. 518.]
  [286. 522.]]]
[INFO] ArUco marker ID: 7
[[[157. 398.]
  [269. 409.]
  [272. 522.]
  [157. 518.]]]
[INFO] ArUco marke

[[[378. 402.]
  [505. 401.]
  [506. 530.]
  [378. 531.]]]
[INFO] ArUco marker ID: 8
[[[102. 403.]
  [225. 403.]
  [223. 530.]
  [ 98. 529.]]]
[INFO] ArUco marker ID: 6
[[[239. 402.]
  [364. 402.]
  [362. 531.]
  [237. 530.]]]
[INFO] ArUco marker ID: 7
[[[247. 266.]
  [369. 267.]
  [364. 388.]
  [238. 388.]]]
[INFO] ArUco marker ID: 4
[[[113. 267.]
  [234. 267.]
  [225. 389.]
  [103. 389.]]]
[INFO] ArUco marker ID: 3
[[[383. 265.]
  [505. 267.]
  [504. 388.]
  [378. 387.]]]
[INFO] ArUco marker ID: 5
[[[136. 152.]
  [253. 157.]
  [235. 254.]
  [115. 255.]]]
[INFO] ArUco marker ID: 0
[[[238. 397.]
  [362. 396.]
  [363. 526.]
  [238. 526.]]]
[INFO] ArUco marker ID: 7
[[[100. 401.]
  [223. 398.]
  [224. 526.]
  [ 98. 525.]]]
[INFO] ArUco marker ID: 6
[[[377. 397.]
  [505. 396.]
  [506. 525.]
  [378. 527.]]]
[INFO] ArUco marker ID: 8
[[[114. 263.]
  [233. 263.]
  [224. 385.]
  [102. 386.]]]
[INFO] ArUco marker ID: 3
[[[247. 262.]
  [368. 263.]
  [363. 384.]
  [238. 384.]]]
[INFO] ArUco marke

[[[ 89. 381.]
  [212. 378.]
  [214. 505.]
  [ 87. 508.]]]
[INFO] ArUco marker ID: 6
[[[226. 378.]
  [352. 375.]
  [354. 504.]
  [227. 505.]]]
[INFO] ArUco marker ID: 7
[[[366. 375.]
  [494. 372.]
  [496. 501.]
  [368. 504.]]]
[INFO] ArUco marker ID: 8
[[[ 95. 243.]
  [215. 239.]
  [213. 364.]
  [ 90. 367.]]]
[INFO] ArUco marker ID: 3
[[[229. 239.]
  [352. 235.]
  [352. 361.]
  [226. 364.]]]
[INFO] ArUco marker ID: 4
[[[366. 235.]
  [490. 231.]
  [493. 358.]
  [366. 361.]]]
[INFO] ArUco marker ID: 5
[[[109. 118.]
  [228. 117.]
  [216. 227.]
  [ 96. 230.]]]
[INFO] ArUco marker ID: 0
[[[240. 117.]
  [359. 115.]
  [352. 223.]
  [230. 226.]]]
[INFO] ArUco marker ID: 1
[[[372. 114.]
  [489. 109.]
  [490. 218.]
  [366. 222.]]]
[INFO] ArUco marker ID: 2
[[[ 89. 382.]
  [212. 379.]
  [214. 506.]
  [ 88. 509.]]]
[INFO] ArUco marker ID: 6
[[[226. 379.]
  [352. 376.]
  [354. 504.]
  [227. 506.]]]
[INFO] ArUco marker ID: 7
[[[366. 376.]
  [494. 373.]
  [496. 502.]
  [367. 505.]]]
[INFO] ArUco marke

...

[[[316. 368.]
  [437. 372.]
  [439. 503.]
  [318. 494.]]]
[INFO] ArUco marker ID: 7
[[[451. 372.]
  [587. 375.]
  [588. 510.]
  [453. 504.]]]
[INFO] ArUco marker ID: 8
[[[201. 363.]
  [304. 368.]
  [305. 494.]
  [195. 485.]]]
[INFO] ArUco marker ID: 6
[[[207. 231.]
  [306. 227.]
  [304. 354.]
  [202. 350.]]]
[INFO] ArUco marker ID: 3
[[[317. 227.]
  [434. 223.]
  [437. 357.]
  [317. 355.]]]
[INFO] ArUco marker ID: 4
[[[449. 223.]
  [584. 220.]
  [587. 359.]
  [451. 358.]]]
[INFO] ArUco marker ID: 5
[[[218. 105.]
  [309.  91.]
  [306. 214.]
  [208. 218.]]]
[INFO] ArUco marker ID: 0
[[[322.  89.]
  [435.  76.]
  [435. 209.]
  [318. 213.]]]
[INFO] ArUco marker ID: 1
[[[448.  75.]
  [584.  65.]
  [585. 205.]
  [449. 209.]]]
[INFO] ArUco marker ID: 2
[[[452. 373.]
  [586. 374.]
  [589. 508.]
  [455. 505.]]]
[INFO] ArUco marker ID: 8
[[[316. 369.]
  [437. 372.]
  [440. 503.]
  [320. 494.]]]
[INFO] ArUco marker ID: 7
[[[202. 364.]
  [304. 369.]
  [306. 494.]
  [198. 485.]]]
[INFO] ArUco marke

[[[451. 378.]
  [588. 379.]
  [590. 515.]
  [455. 511.]]]
[INFO] ArUco marker ID: 8
[[[314. 374.]
  [437. 378.]
  [440. 510.]
  [317. 501.]]]
[INFO] ArUco marker ID: 7
[[[202. 368.]
  [302. 374.]
  [305. 500.]
  [198. 490.]]]
[INFO] ArUco marker ID: 6
[[[313. 230.]
  [432. 227.]
  [436. 364.]
  [313. 360.]]]
[INFO] ArUco marker ID: 4
[[[206. 234.]
  [301. 231.]
  [301. 360.]
  [203. 355.]]]
[INFO] ArUco marker ID: 3
[[[446. 227.]
  [586. 224.]
  [588. 364.]
  [451. 364.]]]
[INFO] ArUco marker ID: 5
[[[214. 105.]
  [302.  90.]
  [302. 216.]
  [208. 222.]]]
[INFO] ArUco marker ID: 0
[[[314.  88.]
  [429.  75.]
  [432. 213.]
  [313. 217.]]]
[INFO] ArUco marker ID: 1
[[[443.  73.]
  [583.  63.]
  [586. 208.]
  [446. 212.]]]
[INFO] ArUco marker ID: 2
[[[451. 379.]
  [588. 379.]
  [591. 514.]
  [455. 511.]]]
[INFO] ArUco marker ID: 8
[[[314. 374.]
  [437. 378.]
  [440. 510.]
  [318. 501.]]]
[INFO] ArUco marker ID: 7
[[[202. 368.]
  [302. 374.]
  [305. 501.]
  [199. 490.]]]
[INFO] ArUco marke

[[[455. 379.]
  [592. 380.]
  [595. 514.]
  [460. 511.]]]
[INFO] ArUco marker ID: 8
[[[319. 375.]
  [441. 379.]
  [445. 510.]
  [323. 501.]]]
[INFO] ArUco marker ID: 7
[[[208. 369.]
  [307. 376.]
  [311. 501.]
  [205. 489.]]]
[INFO] ArUco marker ID: 6
[[[317. 232.]
  [435. 229.]
  [441. 365.]
  [318. 361.]]]
[INFO] ArUco marker ID: 4
[[[211. 236.]
  [305. 233.]
  [307. 360.]
  [209. 356.]]]
[INFO] ArUco marker ID: 3
[[[449. 228.]
  [589. 225.]
  [592. 365.]
  [454. 365.]]]
[INFO] ArUco marker ID: 5
[[[218. 106.]
  [306.  91.]
  [306. 218.]
  [212. 223.]]]
[INFO] ArUco marker ID: 0
[[[317.  90.]
  [431.  76.]
  [435. 214.]
  [318. 219.]]]
[INFO] ArUco marker ID: 1
[[[445.  75.]
  [586.  65.]
  [589. 210.]
  [449. 214.]]]
[INFO] ArUco marker ID: 2
[[[455. 379.]
  [592. 380.]
  [595. 514.]
  [460. 511.]]]
[INFO] ArUco marker ID: 8
[[[319. 375.]
  [441. 379.]
  [445. 510.]
  [323. 501.]]]
[INFO] ArUco marker ID: 7
[[[208. 369.]
  [307. 376.]
  [311. 501.]
  [205. 489.]]]
[INFO] ArUco marke

[[[456. 380.]
  [592. 383.]
  [594. 519.]
  [458. 513.]]]
[INFO] ArUco marker ID: 8
[[[320. 375.]
  [441. 380.]
  [443. 512.]
  [323. 500.]]]
[INFO] ArUco marker ID: 7
[[[209. 368.]
  [308. 374.]
  [311. 500.]
  [205. 489.]]]
[INFO] ArUco marker ID: 6
[[[320. 232.]
  [438. 230.]
  [441. 365.]
  [320. 361.]]]
[INFO] ArUco marker ID: 4
[[[214. 235.]
  [308. 232.]
  [308. 360.]
  [209. 354.]]]
[INFO] ArUco marker ID: 3
[[[452. 229.]
  [591. 227.]
  [592. 368.]
  [456. 366.]]]
[INFO] ArUco marker ID: 5
[[[221. 106.]
  [310.  92.]
  [309. 219.]
  [214. 222.]]]
[INFO] ArUco marker ID: 0
[[[321.  90.]
  [436.  78.]
  [438. 215.]
  [320. 218.]]]
[INFO] ArUco marker ID: 1
[[[449.  77.]
  [589.  67.]
  [591. 212.]
  [452. 215.]]]
[INFO] ArUco marker ID: 2
[[[456. 380.]
  [592. 383.]
  [594. 519.]
  [458. 512.]]]
[INFO] ArUco marker ID: 8
[[[320. 374.]
  [442. 381.]
  [444. 511.]
  [323. 500.]]]
[INFO] ArUco marker ID: 7
[[[208. 368.]
  [308. 374.]
  [311. 500.]
  [204. 488.]]]
[INFO] ArUco marke

[[[458. 390.]
  [594. 398.]
  [591. 539.]
  [454. 525.]]]
[INFO] ArUco marker ID: 8
[[[323. 381.]
  [444. 390.]
  [439. 525.]
  [321. 509.]]]
[INFO] ArUco marker ID: 7
[[[205. 374.]
  [311. 381.]
  [308. 508.]
  [197. 497.]]]
[INFO] ArUco marker ID: 6
[[[214. 240.]
  [317. 240.]
  [311. 366.]
  [206. 360.]]]
[INFO] ArUco marker ID: 3
[[[329. 239.]
  [446. 239.]
  [443. 375.]
  [323. 367.]]]
[INFO] ArUco marker ID: 4
[[[461. 239.]
  [597. 241.]
  [594. 383.]
  [458. 376.]]]
[INFO] ArUco marker ID: 5
[[[230. 113.]
  [325. 103.]
  [318. 226.]
  [216. 227.]]]
[INFO] ArUco marker ID: 0
[[[337. 102.]
  [450.  91.]
  [447. 225.]
  [329. 226.]]]
[INFO] ArUco marker ID: 1
[[[457. 392.]
  [594. 400.]
  [591. 541.]
  [453. 529.]]]
[INFO] ArUco marker ID: 8
[[[322. 383.]
  [442. 392.]
  [439. 527.]
  [319. 512.]]]
[INFO] ArUco marker ID: 7
[[[204. 375.]
  [310. 383.]
  [305. 511.]
  [195. 499.]]]
[INFO] ArUco marker ID: 6
[[[461. 240.]
  [596. 242.]
  [594. 385.]
  [457. 378.]]]
[INFO] ArUco marke